<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Загрузка-данных" data-toc-modified-id="Загрузка-данных-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Загрузка данных</a></span></li><li><span><a href="#Умножение-матриц" data-toc-modified-id="Умножение-матриц-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Умножение матриц</a></span></li><li><span><a href="#Алгоритм-преобразования" data-toc-modified-id="Алгоритм-преобразования-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Алгоритм преобразования</a></span></li><li><span><a href="#Проверка-алгоритма" data-toc-modified-id="Проверка-алгоритма-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Проверка алгоритма</a></span></li></ul></div>

# Защита персональных данных клиентов

Нам нужно защитить данные клиентов страховой компании «Хоть потоп» и разработать такой метод преобразования данных, чтобы по ним было сложно восстановить персональную информацию. Обоснуйте корректность его работы.

Нужно защитить данные, чтобы при преобразовании качество моделей машинного обучения не ухудшилось. 

## Загрузка данных

In [2]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score

In [3]:
data = pd.read_csv('/datasets/insurance.csv')

In [4]:
data.head()

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
0,1,41.0,49600.0,1,0
1,0,46.0,38000.0,1,1
2,0,29.0,21000.0,0,0
3,0,21.0,41700.0,2,0
4,1,28.0,26100.0,0,0


In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Пол                5000 non-null   int64  
 1   Возраст            5000 non-null   float64
 2   Зарплата           5000 non-null   float64
 3   Члены семьи        5000 non-null   int64  
 4   Страховые выплаты  5000 non-null   int64  
dtypes: float64(2), int64(3)
memory usage: 195.4 KB


Таблица заполнена полностью, в данных отсутствуют пропуски, форматы столбцов числовые, что позволяет произвести дальнейшее обучение модели и проверку методов шифрования.

## Умножение матриц

Обозначения:

- $X$ — матрица признаков (нулевой столбец состоит из единиц)

- $y$ — вектор целевого признака

- $P$ — матрица, на которую умножаются признаки

- $w$ — вектор весов линейной регрессии (нулевой элемент равен сдвигу)

Предсказания:

$$
a = Xw
$$

Задача обучения:

$$
w = \arg\min_w MSE(Xw, y)
$$

Формула обучения:

$$
w = (X^T X)^{-1} X^T y
$$

Докажем, что $X = XP$

Напишем формулу обучения для произведения признаков на обратимую матрицу: 

$$ w_1 = ((XP)^T XP)^{-1} (XP)^T y $$

Преобразуем далее по свойствам матриц $(XP)^T = P^TX^T$:

$$ w_1 = (P^TX^T XP)^{-1} P^TX^T y $$

Разложим произведение в -1 степени:


$$ w_1 = P^{-1}(X^TX)^{-1}(P^T)^{-1}P^TX^T y $$

Далее $ (P^T)^{-1}P^T = E $ выражение равно единичной матрице, которая ничего не меняет:

$$ w_1 = P^{-1}(X^TX)^{-1} EX^T y $$

Исходя из того, что $ w = (X^T X)^{-1} X^T y $ делаем следующий вывод:

$$ w_1 = P^{-1} w $$

Согласно формуле предсказание равно $ a = Xw $, а следовательно, если мы считаем предикт на XP вместо X то:

$$ a_1 = XPw_1 $$

или

$$ a_1 = XPP^{-1} w = X E w = X w = a $$

Соответсвенно:

$$ X = XP $$

Метод позволит зашифровать данные без потери качества модели.

In [6]:
# Определим признаки и целевой признак
X = data.drop('Страховые выплаты', axis=1).values
y = data['Страховые выплаты'].values

In [7]:
X.shape

(5000, 4)

In [8]:
X

array([[1.00e+00, 4.10e+01, 4.96e+04, 1.00e+00],
       [0.00e+00, 4.60e+01, 3.80e+04, 1.00e+00],
       [0.00e+00, 2.90e+01, 2.10e+04, 0.00e+00],
       ...,
       [0.00e+00, 2.00e+01, 3.39e+04, 2.00e+00],
       [1.00e+00, 2.20e+01, 3.27e+04, 3.00e+00],
       [1.00e+00, 2.80e+01, 4.06e+04, 1.00e+00]])

In [10]:
# Рассчитаем R2 на неизмененных прихнаках
regression = LinearRegression()
regression.fit(X, y)
prediction = regression.predict(X)
r2_score(y, prediction)

0.4249455028666801

In [11]:
# сгенерируем обратимую матрицу
a = np.array(np.random.random((4, 4)))
#X_1 = np.concatenate((np.ones((X.shape[0], 1)), X), axis=1)

In [12]:
# Обучим и оценим модель на произведении матриц
regression_2 = LinearRegression()
regression_2.fit(X@a, y)
prediction_2 = regression_2.predict(X@a)
r2_score(y, prediction_2)

0.424945502866679

**Ответ:** b. Не изменится

**Обоснование:** показатель R_2 не показал разницу в результатах.

## Алгоритм преобразования

Как показали результаты предыдущего пункта, умножение на обратимую матрицу не влияет на результаты предсказания, поэтому предлагается признаки умножить на случайно сгенерированную обратимую матрицу.

In [14]:
Z = X@a

In [15]:
data_2 = pd.DataFrame(Z, columns=data.drop('Страховые выплаты', axis=1).columns)

Напишем функцию для преобразования

In [16]:
def alhorytm (X):
    Z = X @ np.array(np.random.random((X.shape[1], X.shape[1])))
    return Z

In [17]:
data.head()

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
0,1,41.0,49600.0,1,0
1,0,46.0,38000.0,1,1
2,0,29.0,21000.0,0,0
3,0,21.0,41700.0,2,0
4,1,28.0,26100.0,0,0


In [18]:
data_2 = pd.DataFrame(alhorytm(X), columns=data.drop('Страховые выплаты', axis=1).columns)
data_2['Страховые выплаты'] = data['Страховые выплаты']
data_2.head()

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
0,41707.711438,25671.180376,30392.063870,13985.854784,0
1,31953.923147,19671.740567,23296.660978,10724.265127,1
2,17658.434873,10871.871836,12877.460039,5928.814659,0
3,35064.206505,21578.546262,25540.017315,11749.410095,0
4,21947.356922,13510.350703,15998.029543,7363.744849,0


**Алгоритм**
Умножение на обратимую матрицу

**Обоснование**

Данные зашифрованы как и полагается

## Проверка алгоритма

In [28]:
# Рассчитаем R2 на неизмененных прихнаках
regression = LinearRegression()
regression.fit(X, y)
prediction = regression.predict(X)
r2_score(y, prediction)

0.4249455028666801

In [20]:
# Определим признаки и целевой признак
X_2 = data_2.drop('Страховые выплаты', axis=1).values

In [29]:
# Рассчитаем R2 на неизмененных прихнаках
regression_2 = LinearRegression()
regression_2.fit(X_2, y)
prediction_2 = regression_2.predict(X_2)
r2_score(y, prediction_2)

0.42494550286668065

In [22]:
class LinearRegressionMine:
    def fit(self, train_features, train_target):
        X = np.concatenate((np.ones((train_features.shape[0], 1)), train_features), axis=1)
        y = train_target
        w = np.linalg.inv(X.T.dot(X)).dot(X.T).dot(y)
        self.w = w[1:]
        self.w0 = w[0]

    def predict(self, test_features):
        return test_features.dot(self.w) + self.w0

In [23]:
model = LinearRegressionMine()
model.fit(X, y)
predictions = model.predict(X)
print(r2_score(y, predictions))

0.42494550286668


In [24]:
model = LinearRegressionMine()
model.fit(Z, y)
predictions = model.predict(Z)
print(r2_score(y, predictions))

0.4249455028666781


Показатель снова сходится